In [92]:
import sys
import os
import pandas as pd
import tiktoken
from tenacity import (
    retry,
    stop_after_attempt,
    wait_random_exponential,
)  # for exponential backoff

#sys.path.append('/home/vs428/Documents/DischargeMe/hail-dischargeme/scoring/scoring.py')

In [235]:
import re

In [93]:
%load_ext dotenv
%dotenv /vast/palmer/home.mccleary/vs428/Documents/DischargeMe/hail-dischargeme/.env

The dotenv extension is already loaded. To reload it, use:
  %reload_ext dotenv


In [94]:
def num_tokens_from_string(string: str, encoding_name: str="cl100k_base") -> int:
    """Returns the number of tokens in a text string."""
    encoding = tiktoken.get_encoding(encoding_name)
    num_tokens = len(encoding.encode(string))
    return num_tokens

In [95]:
@retry(wait=wait_random_exponential(min=1, max=60), stop=stop_after_attempt(6))
def completion_with_backoff(**kwargs):
    return openai.ChatCompletion.create(**kwargs)


In [96]:
from IPython.display import display, HTML

def add_line_breaks(text):
    return text.replace('\n', '<br>')


def pretty_print(df):
    return display( HTML( df.to_html().replace("\\n","<br>") ) )

In [97]:
challenge_data_fp = "/gpfs/gibbs/project/rtaylor/shared/DischargeMe/public/"


In [98]:
from datasets import load_dataset

#train_dataset = load_dataset('json', data_files=challenge_data_fp + "train/simple_train.json", split='train')
#eval_dataset = load_dataset('json', data_files=challenge_data_fp + "valid/simple_valid.json", split='train')
test_dataset = load_dataset('json', data_files="/home/vs428/Documents/DischargeMe/hail-dischargeme/notebooks/data_processing/simple_test.json", split="train")

Generating train split: 0 examples [00:00, ? examples/s]

In [99]:
#eval_dataset = eval_dataset.select(range(500))

In [100]:
import openai
openai.api_type = "azure"
openai.api_base = os.getenv("AZURE_OPENAI_ENDPOINT")
openai.api_version = "2023-07-01-preview"
openai.api_key = os.getenv("AZURE_OPENAI_KEY")
engine = "decile-gpt-35-turbo-16k"

In [101]:
gpt_inputs = []
for datum in test_dataset:
    message_text = [{"role":"system","content":"You are a physician generating a summary brief hospital course from the patient encounter information given"}]
    
    gpt_service_prompt = {"role":"user",
                     "content":f"Summarize the following patient hospital encounter into a brief hospital course:\n\n{datum['input']}" }
    
    message_text.append(gpt_service_prompt)
    
    # print(f"Deprescribe Prompt: {message_text}")
    gpt_inputs.append(message_text)

In [102]:
len(gpt_inputs)

14702

In [103]:
completions = []
for idx, gpt_input in enumerate(gpt_inputs[:10]):
    completion = openai.ChatCompletion.create(
      engine=engine,
      messages = gpt_input,
    )
    print(idx)
    completions.append(completion['choices'][0]['message']['content'])

0
1
2
3
4
5
6
7
8
9


In [104]:
completions

['The patient, a male with a history of HCV, ETOH cirrhosis, lymphedema, and COPD, presented with increasing jaundice, abdominal pain, and distension. On examination, he had tense ascites, diffuse tenderness throughout the abdomen, and crackles heard on lung auscultation. Imaging studies showed a shrunken cirrhotic liver, a liver mass, and moderate ascites. Laboratory results were significant for low hemoglobin and platelet counts, and elevated liver enzymes. The patient underwent a paracentesis, which revealed no evidence of spontaneous bacterial peritonitis. Microbiological cultures were negative. He was discharged with ongoing symptoms of abdominal pain and difficulty breathing.  Further evaluation with an MRI was recommended to assess the liver mass.',
 "Patient hospital encounter summary:\n\nThe patient is a middle-aged female who was admitted to the Medicine service with a chief complaint of bleeding from her surgical wound. She had previously undergone a L4-5 discectomy/fusion a

In [105]:
from rouge_score import rouge_scorer

scorer = rouge_scorer.RougeScorer(['rouge1', "rouge2", 'rougeL'], use_stemmer=True)


In [106]:
all_scores = []
for gen, ref in zip(completions, test_dataset['output'][:10]):
    scores = scorer.score(gen, ref)
    scores_dict = {"rouge1_precision":scores['rouge1'][0], 
                  "rouge1_recall":scores['rouge1'][1], 
                  "rouge1_f1":scores['rouge1'][2], 
                  "rouge2_precision":scores['rouge2'][0], 
                  "rouge2_recall":scores['rouge2'][1], 
                  "rouge2_f1":scores['rouge2'][2], 
                  "rougeL_precision":scores['rougeL'][0], 
                  "rougeL_recall":scores['rougeL'][1], 
                  "rougeL_f1":scores['rougeL'][2], }
    all_scores.append(scores_dict)

In [107]:
comparison_df = pd.DataFrame([completions, test_dataset['output'][:10]]).T.rename({0:"GPT", 1:"gold-standard"}, axis=1)

In [108]:
comparison_df_styled = comparison_df.style.format({'completion_text': add_line_breaks})


,GPT,gold-standard
0,"The patient, a male with a history of HCV, ETO...","___ with PMH HCV, ETOH cirrhosis with ascites,..."


In [117]:
with pd.option_context('display.max_colwidth', None):
    pretty_print(comparison_df.iloc[[0]]
)

,GPT,gold-standard
0,"The patient, a male with a history of HCV, ETOH cirrhosis, lymphedema, and COPD, presented with increasing jaundice, abdominal pain, and distension. On examination, he had tense ascites, diffuse tenderness throughout the abdomen, and crackles heard on lung auscultation. Imaging studies showed a shrunken cirrhotic liver, a liver mass, and moderate ascites. Laboratory results were significant for low hemoglobin and platelet counts, and elevated liver enzymes. The patient underwent a paracentesis, which revealed no evidence of spontaneous bacterial peritonitis. Microbiological cultures were negative. He was discharged with ongoing symptoms of abdominal pain and difficulty breathing. Further evaluation with an MRI was recommended to assess the liver mass.","___ with PMH HCV, ETOH cirrhosis with ascites, hx of DVT, lymphedema, COPD, hx of MRSA bacteremia s/p 4 week course of daptomycin (completed ___ transfer from ___ after presenting with 2 weeks of worsening abdominal distention and pain as well as worsening jaundice found to have decompensated cirrhosis, ___, and a new liver lesion very concerning for ___, discharged to hospice."


In [110]:
# with open(f"comparison_simple_gpt3.5.html", "w") as file:
#     file.write(comparison_df_styled.to_html())

In [111]:
import pandas as pd
pd.DataFrame.from_records(all_scores).mean()

rouge1_precision    0.264161
rouge1_recall       0.390879
rouge1_f1           0.298986
rouge2_precision    0.051340
rouge2_recall       0.076467
rouge2_f1           0.058354
rougeL_precision    0.129027
rougeL_recall       0.179565
rougeL_f1           0.141151
dtype: float64

# N-shot

In [451]:
# examples = ["Example\n######\n\n" + ex for ex in eval_dataset['output'][5:7]]
ex1 = """Mr. ___ presented to the ED s/p mechanical fall with head 
strike. CTH at OSH with subacute right SDH for which patient was 
transferred to ___ for evaluation and neurosurgery was 
consulted on arrival to ___ ED. 

#Right SDH
Patient remained neurologically at his baseline in the ED. He 
was admitted to the neurosurgery service and transferred to the 
floor for continued neurological monitoring. Patient's INR on 
admission was 1.5 and he was given Vitamin K 10mg IV x1 and then 
vitamin K 5mg x2 for a total of 3 doses. Patient's INR was 
monitored closely during hospitalization and down trended 
appropriately. Repeat CTH showed a stable SDH size. Patient 
remained stable neurologically. He remained on TF as he was at 
home and will follow up with SLP and laryngologist outpatient. 
While hospitalized patient remained at his neurological 
baseline. 

#Hx colloid cyst rupture s/p bilateral VPS placement
On patient's initial and repeat NCHCTs, his ventricles were 
shown to be slit. His bilateral VPS were turned up from 1.0 to 
1.5. No scan was repeated following adjustment of his VPS 
settings. He remained neurologically stable following this 
change. 

#Hx of DVT
Patient with a history of bilateral femoral vein occlusive DVTs 
for which he was started on Eliquis. Due to this subacute SDH 
Eliquis was held and bilateral LENIs were obtained to assess 
DVTs. These were negative for thrombus and the Eliquis was 
stopped indefinitely. Patient was placed on ___ 24hrs s/p 
trauma. 

#Disposition
Patient was evaluated by physical therapy who recommended 
___ rehab. Patient was discharged to rehab on ___. 
Patient discharged with follow up instructions with Dr. ___ in 
4 weeks with a repeat non contrast CTH"""

ex2 = """Mr. ___ is a ___ y/o man w/ HTN who presented w/ sudden-onset LUE 
weakness ___ AM). Next day, presented to ED. Pt noted to have 
L facial droop, LUE weakness (more prominent distally) and 
decreased LUE sensation. CTA showed significant R ICA stenosis; 
U/S indicated occlusion between 80 and 100%. MRI brain showed 
acute small infarcts in R frontal and parietal lobes in the MCA 
territory, consistent w/ watershed infarcts. Started on ASA and 
clopidogrel."""

examples = [ex1, ex2]

In [ ]:
gpt_inputs = []
for datum in eval_dataset:
    message_text = [{"role":"system","content":"You are a physician generating a summary brief hospital course from the patient encounter information given"}]
    
    gpt_service_prompt = {"role":"user",
                     "content":f"Summarize the following patient hospital encounter into a brief hospital course:\n\n{datum['input']}.\n\nUse these two as examples:\n\n{'\n\n'.join(examples)}" }
    
    message_text.append(gpt_service_prompt)
    
    # print(f"Deprescribe Prompt: {message_text}")
    gpt_inputs.append(message_text)

In [ ]:
# (((num_tokens_from_string(gpt_inputs[123][1]['content'], "cl100k_base")) / 1000 * 0.01) + (400 / 1000 * 0.03)) * 10_000

In [ ]:
len(gpt_inputs)

In [ ]:
print(gpt_inputs[0][1]['content'])

In [ ]:
completions = []
for idx, gpt_input in enumerate(gpt_inputs[:10]):
    completion_with_backoff(
      engine=engine,
      messages = gpt_input,        
    )
    print(idx)
    completions.append(completion['choices'][0]['message']['content'])

In [ ]:
completions

In [ ]:
from rouge_score import rouge_scorer

scorer = rouge_scorer.RougeScorer(['rouge1', "rouge2", 'rougeL'], use_stemmer=True)


In [ ]:
all_scores = []
for gen, ref in zip(completions, eval_dataset['output'][:10]):
    scores = scorer.score(gen, ref)
    print(scores)
    scores_dict = {"rouge1_precision":scores['rouge1'][0], 
                  "rouge1_recall":scores['rouge1'][1], 
                  "rouge1_f1":scores['rouge1'][2], 
                  "rouge2_precision":scores['rouge2'][0], 
                  "rouge2_recall":scores['rouge2'][1], 
                  "rouge2_f1":scores['rouge2'][2], 
                  "rougeL_precision":scores['rougeL'][0], 
                  "rougeL_recall":scores['rougeL'][1], 
                  "rougeL_f1":scores['rougeL'][2], }
    all_scores.append(scores_dict)

In [ ]:

def add_line_breaks(text):
    return text.replace('\n', '<br>')


In [ ]:
comparison_df = pd.DataFrame([completions, eval_dataset['output'][:10]]).T.rename({0:"GPT", 1:"gold-standard"}, axis=1)

In [ ]:
comparison_df_styled = comparison_df.style.format({'completion_text': add_line_breaks})


In [ ]:

with open(f"comparison_simple_gpt3.5_2shot.html", "w") as file:
    file.write(comparison_df_styled.to_html())

In [ ]:
import pandas as pd
pd.DataFrame.from_records(all_scores).describe()

# Integrate Service/Ward info

This section will integrate the service-file to match the structure of the note as much as possible. 

In [118]:
target_test = pd.read_csv(challenge_data_fp + "test_phase_1/discharge_target.csv.gz", keep_default_na=False)


In [68]:
# ward transfers
transfers = pd.read_pickle('/gpfs/gibbs/project/rtaylor/shared/DischargeMe/mimiciv/hosp/cohort_transfers.pkl')

# higher-level services (ICU, CARD, etc)
services = pd.read_pickle('/gpfs/gibbs/project/rtaylor/shared/DischargeMe/mimiciv/hosp/cohort_services.pkl')


In [84]:
transfers = transfers[transfers['eventtype'] != "discharge"]

In [85]:
transfers = transfers.sort_values(['hadm_id', "intime"])

In [220]:
discharging_transfer = transfers.groupby("hadm_id").last().reset_index()

In [221]:
discharging_transfer['careunit'].value_counts()

careunit
Medicine                                            59400
Med/Surg                                            20460
Emergency Department Observation                    20027
Medicine/Cardiology                                 15441
Neurology                                           14445
Hematology/Oncology                                 13244
Transplant                                          12433
Vascular                                            10642
Med/Surg/Trauma                                     10496
Med/Surg/GYN                                         9831
Hematology/Oncology Intermediate                     4937
Surgery/Trauma                                       4525
Cardiac Surgery                                      3442
Surgery                                              2980
Medicine/Cardiology Intermediate                     2626
Psychiatry                                           1823
Medical/Surgical (Gynecology)                        1684
Obser

In [222]:
discharging_service = services.sort_values(['hadm_id', "transfertime"]).groupby("hadm_id").last().reset_index()

In [233]:
test_dataset_df = test_dataset.to_pandas(); test_dataset_df.shape

(14702, 4)

In [234]:
test_dataset_df = test_dataset_df.merge(discharging_transfer[['hadm_id', 'careunit', 'eventtype']], on="hadm_id", how="left"); test_dataset_df.shape

(14702, 6)

In [237]:
test_dataset_df = test_dataset_df.merge(discharging_service[['hadm_id', 'curr_service']], on="hadm_id", how="left"); test_dataset_df

,note_id,hadm_id,input,output,careunit,eventtype,curr_service
0,19766998-DS-20,26231944,\nName: ___ Unit No: ___\n ...,"___ with PMH HCV, ETOH cirrhosis with ascites,...",Med/Surg,transfer,MED
1,10336082-DS-11,28542384,\nName: ___ Unit No: ___\n \nA...,___ yo F with recent lumbar laminectomy c/b MS...,Neurology,admit,MED
2,10481170-DS-22,26489329,\nName: ___ Unit No: ___\n ...,The patient was seen ___ the emergency departm...,Med/Surg/GYN,transfer,SURG
3,11576109-DS-14,22641254,\nName: ___ Unit No: ___\n ...,"Ms. ___ is a ___ woman with a history of T2DM,...",Medicine/Cardiology,transfer,CMED
4,19249697-DS-17,29265750,\nName: ___ Unit No: ...,___ with IDDM and h/o provoked PE on apixiban ...,Vascular,admit,MED
...,...,...,...,...,...,...,...
14697,18995174-DS-17,27445071,\nName: ___ Unit No: ___\...,___ M PMHx dilated non-ischemic cardiomyopathy...,Medicine/Cardiology,admit,CMED
14698,13588195-DS-9,26192891,\nName: ___ Unit No: ___\n ...,"Mr. ___ is a ___ year-old man with CAD, HTN, H...",Med/Surg,admit,MED
14699,10873131-DS-17,26584893,\nName: ___ Unit No: _...,___ year old female with past medical history ...,Medicine,transfer,MED
14700,12332377-DS-11,25623241,\nName: ___ Unit No: ___\n ...,Mr. ___ is a ___ male with a past medical hist...,Med/Surg/Trauma,admit,MED


In [241]:
test_dataset_df['curr_service'].value_counts()

curr_service
MED      7616
CMED     1340
NMED     1333
SURG     1293
OMED      883
ORTHO     699
NSURG     481
TRAUM     340
VSURG     210
CSURG     132
GYN        99
GU         95
TSURG      87
PSURG      42
OBS        38
PSYCH       8
ENT         6
Name: count, dtype: int64

In [202]:
# print(test_dataset_df[test_dataset_df['careunit'] == "Med/Surg"].sample(1)['output'].squeeze())

test_dataset_df[test_dataset_df['careunit'] == "Hematology/Oncology"]['output'].str.contains("ACUTE ISSUES", case=False).value_counts()

output
False    697
True      17
Name: count, dtype: int64

In [494]:
test_dataset_df

,note_id,hadm_id,input,output,careunit,eventtype,curr_service
0,19766998-DS-20,26231944,\nName: ___ Unit No: ___\n ...,"___ with PMH HCV, ETOH cirrhosis with ascites,...",Med/Surg,transfer,MED
1,10336082-DS-11,28542384,\nName: ___ Unit No: ___\n \nA...,___ yo F with recent lumbar laminectomy c/b MS...,Neurology,admit,MED
2,10481170-DS-22,26489329,\nName: ___ Unit No: ___\n ...,The patient was seen ___ the emergency departm...,Med/Surg/GYN,transfer,SURG
3,11576109-DS-14,22641254,\nName: ___ Unit No: ___\n ...,"Ms. ___ is a ___ woman with a history of T2DM,...",Medicine/Cardiology,transfer,CMED
4,19249697-DS-17,29265750,\nName: ___ Unit No: ...,___ with IDDM and h/o provoked PE on apixiban ...,Vascular,admit,MED
...,...,...,...,...,...,...,...
14697,18995174-DS-17,27445071,\nName: ___ Unit No: ___\...,___ M PMHx dilated non-ischemic cardiomyopathy...,Medicine/Cardiology,admit,CMED
14698,13588195-DS-9,26192891,\nName: ___ Unit No: ___\n ...,"Mr. ___ is a ___ year-old man with CAD, HTN, H...",Med/Surg,admit,MED
14699,10873131-DS-17,26584893,\nName: ___ Unit No: _...,___ year old female with past medical history ...,Medicine,transfer,MED
14700,12332377-DS-11,25623241,\nName: ___ Unit No: ___\n ...,Mr. ___ is a ___ male with a past medical hist...,Med/Surg/Trauma,admit,MED


In [493]:
print(test_dataset_df[test_dataset_df['curr_service'] == "SURG"]['output'].sample(1).squeeze())

The patient's CT Abd/Pelvis in the ED was negative for an 
internal hernia. She was admitted to the ___ service under 
Dr ___.

Neuro: The patient was alert and oriented throughout 
hospitalization; pain was initially managed with IV pain 
medication and then transitioned to PO pain medications once 
patient was tolerating a stage 2 diet. 

CV: The patient remained stable from a cardiovascular 
standpoint; vital signs were routinely monitored.

Pulmonary: The patient remained stable from a pulmonary 
standpoint; vital signs were routinely monitored. Good pulmonary 
toilet, early ambulation and incentive spirometry were 
encouraged throughout hospitalization.
 
GI/GU/FEN: The patient was initially kept NPO. She was initially 
kept NPO and started on a banana bag. Full work up including 
nutrition labs were sent and the patient was started on a 
PPI/Carafate. A RUQ ultrasound was negative for gallstones or 
cholecystits and an EGD was normal up to the jejunojejunal 
anastomosis with no e

In [400]:
test_dataset_df.groupby("curr_service").agg({"output":lambda x: x.str.contains("#").sum()/x.shape[0]}).sort_values('output', ascending=False)

,output
curr_service,
OMED,0.768969
CMED,0.752985
PSYCH,0.750000
MED,0.721376
ORTHO,0.686695
PSURG,0.547619
OBS,0.421053
CSURG,0.363636
NMED,0.363091


In [421]:
test_dataset_df.groupby("careunit").agg({"output":lambda x: x.str.contains("#").sum()}).sort_values('output', ascending=False)

,output
careunit,
Medicine,3099
Med/Surg,1116
Medicine/Cardiology,750
Neurology,562
Hematology/Oncology,555
Med/Surg/Trauma,500
Transplant,457
Med/Surg/GYN,394
Vascular,367


In [420]:
test_dataset_df.groupby("careunit").agg({"output":lambda x: x.str.contains("#").sum()/x.shape[0]}).sort_values('output', ascending=False).loc[test_dataset_df.groupby("careunit").agg({"output":lambda x: x.str.contains("#").sum()}).sort_values('output', ascending=False).index]

,output
careunit,
Medicine,0.665450
Med/Surg,0.595836
Medicine/Cardiology,0.746269
Neurology,0.374168
Hematology/Oncology,0.777311
Med/Surg/Trauma,0.465116
Transplant,0.604497
Med/Surg/GYN,0.617555
Vascular,0.558600


In [450]:
print(test_dataset_df[test_dataset_df['careunit'] == "Medicine"]['output'].sample(1).squeeze())

Patient admission summary:
Mr. ___ is a ___ y/o man w/ HTN who presented w/ sudden-onset LUE 
weakness ___ AM). Next day, presented to ED. Pt noted to have 
L facial droop, LUE weakness (more prominent distally) and 
decreased LUE sensation. CTA showed significant R ICA stenosis; 
U/S indicated occlusion between 80 and 100%. MRI brain showed 
acute small infarcts in R frontal and parietal lobes in the MCA 
territory, consistent w/ watershed infarcts. Started on ASA and 
clopidogrel.
